# DUSP1 Confirmation and Visualization Notebook

This notebook processes experimental data from Big-FISH and CellProperties CSV files to classify and visualize DUSP1 smiFISH spots. The analysis is modular, using dedicated classes for loading data, performing signal-to-noise classification, measurement extraction, and filtering.

---

### **Input**
- Big-FISH CSV files (`spots`, `clusters`)
- CellProperties CSV files (`cell_props`, `cell_results`)

---

### **Workflow Overview**

1. **Load Experimental Data**
   - Use `DUSP1AnalysisManager` to identify and load datasets from HDF5 files or local CSVs.
   - Extract file paths from a log directory if not directly provided.

2. **Classify Spots by Signal Quality (SNR Analysis)**
   - Use `SNRAnalysis` to perform:
     - **Weighted SNR thresholding** using Big-FISH `'snr'` values with a percentile cutoff (e.g., 20th percentile within the range 2–5).
     - **Absolute thresholding** using a fixed cutoff on the Big-FISH `'snr'` value.
     - **MG SNR calculation**:  
       A more localized method that accounts for subcellular context. It computes:
       ```
       MG_SNR = (signal - mean) / std
       ```
       where `mean` and `std` are drawn from either the nuclear or cytoplasmic region of the same cell depending on the spot's location. This method reflects more accurate signal variation within individual cells.

   - Adds boolean flags and comparison columns (`MG_pass`, `Abs_pass`, `Weighted_pass`) to aid classification.

3. **Spot and Cell Measurement Extraction**
   - Use `DUSP1Measurement` to:
     - Quantify spot-level and cell-level metrics.
     - Support optional filtering using SNR results.
     - Append `unique_cell_id` for downstream aggregation and visualization.

4. **Filter and Save Processed Data**
   - Apply quality filters to retain only confident spots and cells.
   - Final outputs include:
     - `Finalspots`
     - `Finalclusters`
     - `Finalcellprops`
     - `SSITcellresults`

5. **Visualization**
   - Use `DUSP1DisplayManager` to inspect and validate each analysis step.
   - View raw data, spot overlays, SNR distributions, and per-cell metrics.

---

### **Core Classes**

### **Core Classes**

- **`DUSP1AnalysisManager`**  
  Loads HDF5 datasets or existing CSVs. Manages filepaths and FOV indexing. Exports intermediate and final data to CSV.

- **`SNRAnalysis`**  
  Performs three types of signal quality assessment:  
  1. **Weighted SNR**: percentile-based filtering on Big-FISH `snr`.  
  2. **Absolute SNR**: fixed-threshold filtering on Big-FISH `snr`.  
  3. **MG SNR**: cell-aware SNR using cytoplasmic or nuclear mean & std from CellProperties.  
  Adds boolean flags (`MG_pass`, `Abs_pass`, `Weighted_pass`) and diagnostic columns.

- **`DUSP1Measurement`**  
  Aggregates spot- and cluster-level data to the cell level. Calculates per-cell metrics (e.g. total spots, nuclear vs cytoplasmic counts) and attaches unique IDs for downstream grouping.

- **`DUSP1Filtering`**  
  Applies the various SNR‐based quality filters to spot and cluster data:  
  - Subsets spot tables to only those that pass both absolute and MG SNR checks.  
  - Prunes whole cells if too few spots remain or if cell‐level metrics fall outside expected ranges.  
  - Writes out the cleaned tables:  
    - `Finalspots.csv`  
    - `Finalclusters.csv`  
    - `Finalcellprops.csv`
    - `SSITcellresults.csv`  

- **`DUSP1DisplayManager`**  
  Interactive and static visual tools:  
  - **Segmentation overlays**: show nuclei/cytoplasm masks on raw images.  
  - **Spot detection review**: overlay spot foci on images.  
  - **Crop display**: view spot thumbnails for manual QC.

- **`PostProcessingPlotter`**  
  Statistical summary plots on the final, filtered data:  
  1. **TS‐Frequency Bar Chart**: fraction of cells with 1, 2, 3, or ≥4 transcription sites over time.  
  2. **Ridge (Joy) Plots**: stacked distributions for nuclear, cytoplasmic, and total mRNA counts, with control CDF thresholds annotated.  
  3. **Line Plots (Mean ± SD)**: nuclear and cytoplasmic mRNA counts over time, including the 0 min control baseline, with error bands indicating standard deviation.

---

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.array as da
import os
import sys
import logging
import seaborn as sns
import datetime
from time import sleep

# Today's date
today = datetime.date.today()
# Format date as 'Jun03' (for example)
date_str = today.strftime("%b%d")

logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.Analysis_DUSP1 import DUSP1AnalysisManager, SNRAnalysis, DUSP1Measurement, DUSP1_filtering, DUSP1DisplayManager, PostProcessingPlotter

**`DUSP1AnalysisManager`** 
   - Manages HDF5 file access.
   - Extracts file paths from a log directory (if no direct locations are provided).
   - Provides methods to select an analysis (by name) and load datasets from HDF5 files.
   - Saves datasets as CSV.

In [ ]:
loc = None
log_location = r'/Volumes/share/Users/Eric/GR_DUSP1_reruns'
save_dir = r'/Volumes/share/Users/Eric/GR_DUSP1_AllData/DUSP1_FinalAnalysis_061925'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Define filtering method and thresholds
method = 'mg_abs'            # options: 'mg', 'absolute', 'mg_abs', 'weighted', 'rf', 'none'
mac = True

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.list_analysis_names()

**`Combine Analyses function`**

In [ ]:
def make_combined_manager(loc, log_location, mac, analysis_names):
    """
    Build a single DUSP1AnalysisManager that “knows about” all of the N analyses.
    
    Parameters:
      - loc, log_location, mac: passed straight through to each temp manager
      - analysis_names: list of the analysis_name strings for each analysis
    Returns:
      - a DUSP1AnalysisManager whose .location and .analysis_names
        cover every file + analysis in analysis_names
    """
    all_paths = []
    all_anames = []

    for name in analysis_names:
        temp = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=mac)
        try:
            temp.select_analysis(name)
            # grab its now-filtered list of files + names
            all_paths   .extend(temp.location)
            all_anames  .extend(temp.analysis_names)
        finally:
            temp.close()    # <— guarantee no HDF5 leak
    # now build the combined
    combo = DUSP1AnalysisManager(location=all_paths, mac=mac)
    # manually inject the unioned analysis names
    combo.analysis_names = all_anames
    return combo

**`DUSP1 Replica D 3hr 100nM time-sweep R1 - By Slide`**

In [ ]:
# # ─── 1) pull in all slides  ──────────────────────────────────────────
# analysis_names = [
#     'DUSP1_D_slide1_BFmean_061825',
#     'DUSP1_D_slide2_BFmean_061825',
#     'DUSP1_D_slide3_BFmean_061825',
#     'DUSP1_D_slide4_BFmean_061825',
#     'DUSP1_D_slide5_BFmean_061825',
#     'DUSP1_D_slide6_BFmean_061825',
#     'DUSP1_D_slide7_BFmean_061825',
#     'DUSP1_D_slide8_BFmean_061825',
#     'DUSP1_D_slide9_BFmean_061825',
#     'DUSP1_D_slide10_BFmean_061825',
#     'DUSP1_D_slide11_BFmean_061825',
#     'DUSP1_D_slide12_BFmean_061825',        
# ]

# # build and use the combined manager
# dm = make_combined_manager(loc, log_location, mac, analysis_names)

# spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
# clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
# props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
# dm.close()    # free all HDF5 handles

# print(f"Loaded and concatenated {len(analysis_names)} days:")
# print(f"  spots:   {spots_df.shape}")
# print(f"  clusters:{clusters_df.shape}")
# print(f"  props:   {props_df.shape}")

# # ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
# abs_threshold = 6
# mg_threshold  = 3

# snr = SNRAnalysis(spots_df, props_df, clusters_df,
#                   abs_threshold=abs_threshold,
#                   mg_threshold=mg_threshold)
# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

# dusp = DUSP1Measurement(merged_spots_df,
#                         merged_clusters_df,
#                         merged_cellprops_df)
# cell_level_results = dusp.measure(abs_threshold=abs_threshold,
#                                   mg_threshold=mg_threshold)

# # ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
# max_id     = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))
# rep_prefix = 10
# prefix     = rep_prefix * (10 ** num_digits)

# for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
#     df['unique_cell_id'] += prefix

# # spots
# max_spot_id  = merged_spots_df['unique_spot_id'].max()
# spot_prefix  = rep_prefix ** len(str(max_spot_id))
# merged_spots_df   ['unique_spot_id'] += spot_prefix

# # clusters
# max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # ─── 4) filtering ─────────────────────────────────────────────────────────────
# filterer = DUSP1_filtering(method=method,
#                            abs_threshold=abs_threshold,
#                            mg_threshold=mg_threshold)

# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
#     filterer.apply_all(spots=merged_spots_df,
#                        clusters=merged_clusters_df,
#                        cellprops=merged_cellprops_df)

# # ─── 5) main display ──────────────────────────────────────────────────────────
# # build a fresh manager so the display code can find every HDF5
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# # ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_D'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
# plotter = PostProcessingPlotter(
#     clusters_df=filtered_clusters,
#     cellprops_df=filtered_cellprops,
#     ssit_df=SSITcellresults
# )

# # Time sweep for 100nM Dex
# print("\n>>> Time sweep for 100nM Dex")
# plotter.plot_time_sweep(
#     dex_conc=100,
#     save_dir=output_dir,
#     display=True
# )

**`DUSP1 Replica D 3hr 100nM time-sweep R1 - By Group`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_D_slide1_BFmean_061825',
    'DUSP1_D_slide2_BFmean_061825',
    'DUSP1_D_slide3_BFmean_061825',
    'DUSP1_D_slide4_BFmean_061825',
    'DUSP1_D_slide5_BFmean_061825',
    'DUSP1_D_slide6_BFmean_061825',
    'DUSP1_D_slide7_BFmean_061825',
    'DUSP1_D_slide8_BFmean_061825',
    'DUSP1_D_slide9_BFmean_061825',
    'DUSP1_D_slide10_BFmean_061825',
    'DUSP1_D_slide11_BFmean_061825',
    'DUSP1_D_slide12_BFmean_061825',        
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 10
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# ─── 5) main display ──────────────────────────────────────────────────────────
# build a fresh manager so the display code can find every HDF5
dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
display_manager = DUSP1DisplayManager(
    dm_disp,
    cell_level_results=SSITcellresults,
    spots=filtered_spots,
    clusters=filtered_clusters,
    cellprops=filtered_cellprops,
    removed_spots=removed_spots
)
display_manager.main_display()
dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_D_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults
)

# Time sweep for 100nM Dex
print("\n>>> Time sweep for 100nM Dex")
plotter.plot_time_sweep(
    dex_conc=100,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica D 3hr 100nM time-sweep R1 - Full Dataset`**

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 1) Select & load the “D_Final” analysis
# # ───────────────────────────────────────────────────────────────────────────────
# am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=mac)
# am.select_analysis('DUSP1_D_Final')

# # thresholds
# abs_threshold = 6
# mg_threshold  = 3

# # load all three tables
# spots_df    = am.select_datasets("spotresults",     dtype="dataframe")
# clusters_df = am.select_datasets("clusterresults",  dtype="dataframe")
# props_df    = am.select_datasets("cell_properties", dtype="dataframe")
# am.close()

# # ───────────────────────────────────────────────────────────────────────────────
# # 2) SNRAnalysis → DUSP1Measurement
# # ───────────────────────────────────────────────────────────────────────────────
# snr = SNRAnalysis(spots_df, props_df, clusters_df,
#                   abs_threshold=abs_threshold,
#                   mg_threshold=mg_threshold)
# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

# dusp = DUSP1Measurement(merged_spots_df,
#                         merged_clusters_df,
#                         merged_cellprops_df)
# cell_level_results = dusp.measure(abs_threshold=abs_threshold,
#                                   mg_threshold=mg_threshold)

# # ───────────────────────────────────────────────────────────────────────────────
# # 3) Prefix unique IDs (D runs get rep_prefix=10)
# # ───────────────────────────────────────────────────────────────────────────────
# max_id     = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))
# rep_prefix = 10
# prefix     = rep_prefix * (10 ** num_digits)

# for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
#     df['unique_cell_id'] += prefix

# max_spot_id  = merged_spots_df['unique_spot_id'].max()
# spot_prefix  = rep_prefix ** len(str(max_spot_id))
# merged_spots_df   ['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # ───────────────────────────────────────────────────────────────────────────────
# # 4) Apply filtering → get final tables + SSITcellresults
# # ───────────────────────────────────────────────────────────────────────────────
# filterer = DUSP1_filtering(method=method,
#                            abs_threshold=abs_threshold,
#                            mg_threshold=mg_threshold)
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
#     filterer.apply_all(spots=merged_spots_df,
#                        clusters=merged_clusters_df,
#                        cellprops=merged_cellprops_df)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5) Display image‐based overlays
# # ───────────────────────────────────────────────────────────────────────────────
# dm_disp = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=mac)
# dm_disp.select_analysis('DUSP1_D_Final')
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# # ───────────────────────────────────────────────────────────────────────────────
# # 6) Save final CSVs 
# # ───────────────────────────────────────────────────────────────────────────────
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_D'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSIT.csv"), index=False)
# filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_Spots.csv"), index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_Clusters.csv"), index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_CellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # ───────────────────────────────────────────────────────────────────────────────
# # 7) Post‐processing plots
# # ───────────────────────────────────────────────────────────────────────────────
# plotter = PostProcessingPlotter(
#     clusters_df=filtered_clusters,
#     cellprops_df=filtered_cellprops,
#     ssit_df=SSITcellresults
# )

# # a) time sweep at 100 nM
# plotter.plot_time_sweep(dex_conc=100, save_dir=output_dir, display=True)

**`DUSP1 Replica E 3hr 100nM time-sweep R2 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_E_slide1_BFmean_061825',
    'DUSP1_E_slide2_BFmean_061825',
    'DUSP1_E_slide3_BFmean_061825',
    'DUSP1_E_slide4_BFmean_061825',
    'DUSP1_E_slide5_BFmean_061825',
    'DUSP1_E_slide6_BFmean_061825',
    'DUSP1_E_slide7_BFmean_061825',
    'DUSP1_E_slide8_BFmean_061825',
    'DUSP1_E_slide9_BFmean_061825',
    'DUSP1_E_slide10_BFmean_061825',
    'DUSP1_E_slide11_BFmean_061825',
    'DUSP1_E_slide12_BFmean_061825',        
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 20
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# ─── 5) main display ──────────────────────────────────────────────────────────
# build a fresh manager so the display code can find every HDF5
dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
display_manager = DUSP1DisplayManager(
    dm_disp,
    cell_level_results=SSITcellresults,
    spots=filtered_spots,
    clusters=filtered_clusters,
    cellprops=filtered_cellprops,
    removed_spots=removed_spots
)
display_manager.main_display()
dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_E_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults
)

# Time sweep for 100nM Dex
print("\n>>> Time sweep for 100nM Dex")
plotter.plot_time_sweep(
    dex_conc=100,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica E 3hr 100nM time-sweep R2 - Full Dataset`**

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 1) Select & load the “E_Final” analysis
# # ───────────────────────────────────────────────────────────────────────────────
# am = DUSP1AnalysisManager(location=loc,
#                           log_location=log_location,
#                           mac=mac)
# am.select_analysis('DUSP1_E_Final')

# # thresholds
# abs_threshold = 6
# mg_threshold  = 3

# # load all three tables
# spots_df    = am.select_datasets("spotresults",     dtype="dataframe")
# clusters_df = am.select_datasets("clusterresults",  dtype="dataframe")
# props_df    = am.select_datasets("cell_properties", dtype="dataframe")
# am.close()

# # ───────────────────────────────────────────────────────────────────────────────
# # 2) SNR → DUSP1Measurement
# # ───────────────────────────────────────────────────────────────────────────────
# snr = SNRAnalysis(spots_df, props_df, clusters_df,
#                   abs_threshold=abs_threshold,
#                   mg_threshold=mg_threshold)
# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

# dusp = DUSP1Measurement(merged_spots_df,
#                         merged_clusters_df,
#                         merged_cellprops_df)
# cell_level_results = dusp.measure(abs_threshold=abs_threshold,
#                                   mg_threshold=mg_threshold)

# # ───────────────────────────────────────────────────────────────────────────────
# # 3) Prefix unique IDs (prefix=20 for “E”)
# # ───────────────────────────────────────────────────────────────────────────────
# max_id     = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))
# rep_prefix = 20
# prefix     = rep_prefix * (10 ** num_digits)

# for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
#     df['unique_cell_id'] += prefix

# max_spot_id  = merged_spots_df['unique_spot_id'].max()
# spot_prefix  = rep_prefix ** len(str(max_spot_id))
# merged_spots_df   ['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # ───────────────────────────────────────────────────────────────────────────────
# # 4) Filter → final tables + SSIT
# # ───────────────────────────────────────────────────────────────────────────────
# filterer = DUSP1_filtering(method=method,
#                            abs_threshold=abs_threshold,
#                            mg_threshold=mg_threshold)
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
#     filterer.apply_all(spots=merged_spots_df,
#                        clusters=merged_clusters_df,
#                        cellprops=merged_cellprops_df)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5) Image‐based overlay display
# # ───────────────────────────────────────────────────────────────────────────────
# dm_disp = DUSP1AnalysisManager(location=loc,
#                                log_location=log_location,
#                                mac=mac)
# dm_disp.select_analysis('DUSP1_E_Final')
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# # ───────────────────────────────────────────────────────────────────────────────
# # 6) Save the final, post‐filter CSVs
# # ───────────────────────────────────────────────────────────────────────────────
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_E'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Final results saved to:", output_dir)

# # ───────────────────────────────────────────────────────────────────────────────
# # 7) Post‐processing plots using the new Plotter
# # ───────────────────────────────────────────────────────────────────────────────
# plotter = PostProcessingPlotter(
#     clusters_df=filtered_clusters,
#     cellprops_df=filtered_cellprops,
#     ssit_df=SSITcellresults
# )

# # 7a) Time sweep at one test concentration
# plotter.plot_time_sweep(dex_conc=100, save_dir=output_dir, display=True)

**`DUSP1 Replica F 3hr 100nM time-sweep R3 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_F_slide1_BFmean_061825',
    'DUSP1_F_slide2_BFmean_061825',
    'DUSP1_F_slide3_BFmean_061825',
    'DUSP1_F_slide4_BFmean_061825',
    'DUSP1_F_slide5_BFmean_061825',
    'DUSP1_F_slide6_BFmean_061825',
    'DUSP1_F_slide7_BFmean_061825',
    'DUSP1_F_slide8_BFmean_061825',
    'DUSP1_F_slide9_BFmean_061825',
    'DUSP1_F_slide10_BFmean_061825',
    'DUSP1_F_slide11_BFmean_061825',
    'DUSP1_F_slide12_BFmean_061825',       
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 30
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# ─── 5) main display ──────────────────────────────────────────────────────────
# build a fresh manager so the display code can find every HDF5
dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
display_manager = DUSP1DisplayManager(
    dm_disp,
    cell_level_results=SSITcellresults,
    spots=filtered_spots,
    clusters=filtered_clusters,
    cellprops=filtered_cellprops,
    removed_spots=removed_spots
)
display_manager.main_display()
dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_F_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults
)

# Time sweep for 100nM Dex
print("\n>>> Time sweep for 100nM Dex")
plotter.plot_time_sweep(
    dex_conc=100,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica F 3hr 100nM time-sweep R3 - Full Dataset`**

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 1) Select & load the “F_Final” analysis
# # ───────────────────────────────────────────────────────────────────────────────
# am = DUSP1AnalysisManager(location=loc,
#                           log_location=log_location,
#                           mac=mac)
# am.select_analysis('DUSP1_F_Final')

# # thresholds
# abs_threshold = 6
# mg_threshold  = 3

# # load the raw data tables
# spots_df    = am.select_datasets("spotresults",     dtype="dataframe")
# clusters_df = am.select_datasets("clusterresults",  dtype="dataframe")
# props_df    = am.select_datasets("cell_properties", dtype="dataframe")
# am.close()

# # ───────────────────────────────────────────────────────────────────────────────
# # 2) SNRAnalysis → DUSP1Measurement
# # ───────────────────────────────────────────────────────────────────────────────
# snr = SNRAnalysis(spots_df, props_df, clusters_df,
#                   abs_threshold=abs_threshold,
#                   mg_threshold=mg_threshold)
# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

# dusp = DUSP1Measurement(merged_spots_df,
#                         merged_clusters_df,
#                         merged_cellprops_df)
# cell_level_results = dusp.measure(abs_threshold=abs_threshold,
#                                   mg_threshold=mg_threshold)

# # ───────────────────────────────────────────────────────────────────────────────
# # 3) Prefix unique IDs (prefix=30 for “F”)
# # ───────────────────────────────────────────────────────────────────────────────
# max_id     = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))
# rep_prefix = 30
# prefix     = rep_prefix * (10 ** num_digits)

# for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
#     df['unique_cell_id'] += prefix

# # bump spot and cluster IDs as well
# max_spot_id  = merged_spots_df['unique_spot_id'].max()
# spot_prefix  = rep_prefix ** len(str(max_spot_id))
# merged_spots_df   ['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # ───────────────────────────────────────────────────────────────────────────────
# # 4) Apply filtering → get final tables + SSIT results
# # ───────────────────────────────────────────────────────────────────────────────
# filterer = DUSP1_filtering(method=method,
#                            abs_threshold=abs_threshold,
#                            mg_threshold=mg_threshold)
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
#     filterer.apply_all(spots=merged_spots_df,
#                        clusters=merged_clusters_df,
#                        cellprops=merged_cellprops_df)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5) Display image‐based overlays
# # ───────────────────────────────────────────────────────────────────────────────
# dm_disp = DUSP1AnalysisManager(location=loc,
#                                log_location=log_location,
#                                mac=mac)
# dm_disp.select_analysis('DUSP1_F_Final')
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# # ───────────────────────────────────────────────────────────────────────────────
# # 6) Save only the final, post‐filter CSVs
# # ───────────────────────────────────────────────────────────────────────────────
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_F'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # ───────────────────────────────────────────────────────────────────────────────
# # 7) Post‐processing time sweep plot for this single concentration
# # ───────────────────────────────────────────────────────────────────────────────
# plotter = PostProcessingPlotter(
#     clusters_df=filtered_clusters,
#     cellprops_df=filtered_cellprops,
#     ssit_df=SSITcellresults
# )

# # Time sweep at one test concentration
# test_conc = 100  
# print(f"\n>>> Time sweep @ {test_conc} nM Dex")
# plotter.plot_time_sweep(
#     dex_conc=test_conc,
#     save_dir=output_dir,
#     display=True
# )

**`DUSP1 Replica M 3hr 100nM time-sweep Partial Replica - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_M_slide1_BFmean_061825',
    'DUSP1_M_slide2_BFmean_061825',
    'DUSP1_M_slide3_BFmean_061825',      
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 40
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# ─── 5) main display ──────────────────────────────────────────────────────────
# build a fresh manager so the display code can find every HDF5
dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
display_manager = DUSP1DisplayManager(
    dm_disp,
    cell_level_results=SSITcellresults,
    spots=filtered_spots,
    clusters=filtered_clusters,
    cellprops=filtered_cellprops,
    removed_spots=removed_spots
)
display_manager.main_display()
dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_M_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults
)

# Time sweep for 100nM Dex
print("\n>>> Time sweep for 100nM Dex")
plotter.plot_time_sweep(
    dex_conc=100,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica M 3hr 100nM time-sweep Partial Replica - Full Dataset`**

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 1) Select & load the “M_Final” analysis
# # ───────────────────────────────────────────────────────────────────────────────
# am = DUSP1AnalysisManager(location=loc,
#                           log_location=log_location,
#                           mac=mac)
# am.select_analysis('DUSP1_M_Final')

# # thresholds
# abs_threshold = 6
# mg_threshold  = 3

# # load raw data
# spots_df    = am.select_datasets("spotresults",     dtype="dataframe")
# clusters_df = am.select_datasets("clusterresults",  dtype="dataframe")
# props_df    = am.select_datasets("cell_properties", dtype="dataframe")
# am.close()

# # ───────────────────────────────────────────────────────────────────────────────
# # 2) SNRAnalysis → DUSP1Measurement
# # ───────────────────────────────────────────────────────────────────────────────
# snr = SNRAnalysis(spots_df, props_df, clusters_df,
#                   abs_threshold=abs_threshold,
#                   mg_threshold=mg_threshold)
# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

# dusp = DUSP1Measurement(merged_spots_df,
#                         merged_clusters_df,
#                         merged_cellprops_df)
# cell_level_results = dusp.measure(abs_threshold=abs_threshold,
#                                   mg_threshold=mg_threshold)

# # ───────────────────────────────────────────────────────────────────────────────
# # 3) Prefix unique IDs (prefix=40 for “M”)
# # ───────────────────────────────────────────────────────────────────────────────
# max_id     = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))
# rep_prefix = 40
# prefix     = rep_prefix * (10 ** num_digits)

# for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
#     df['unique_cell_id'] += prefix

# # bump spot & cluster IDs
# max_spot_id  = merged_spots_df['unique_spot_id'].max()
# spot_prefix  = rep_prefix ** len(str(max_spot_id))
# merged_spots_df   ['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # ───────────────────────────────────────────────────────────────────────────────
# # 4) Filtering → final tables + SSIT results
# # ───────────────────────────────────────────────────────────────────────────────
# filterer = DUSP1_filtering(method=method,
#                            abs_threshold=abs_threshold,
#                            mg_threshold=mg_threshold)
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
#     filterer.apply_all(spots=merged_spots_df,
#                        clusters=merged_clusters_df,
#                        cellprops=merged_cellprops_df)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5) Image overlays via DisplayManager
# # ───────────────────────────────────────────────────────────────────────────────
# dm_disp = DUSP1AnalysisManager(location=loc,
#                                log_location=log_location,
#                                mac=mac)
# dm_disp.select_analysis('DUSP1_M_Final')
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# # ───────────────────────────────────────────────────────────────────────────────
# # 6) Save only the final, post‐filter CSVs
# # ───────────────────────────────────────────────────────────────────────────────
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_M'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # ───────────────────────────────────────────────────────────────────────────────
# # 7) Post‐processing: single‐conc (100 nM) time sweep
# # ───────────────────────────────────────────────────────────────────────────────
# plotter = PostProcessingPlotter(
#     clusters_df=filtered_clusters,
#     cellprops_df=filtered_cellprops,
#     ssit_df=SSITcellresults
# )

# test_conc = 100
# print(f"\n>>> Time sweep @ {test_conc} nM Dex")
# plotter.plot_time_sweep(
#     dex_conc=test_conc,
#     save_dir=output_dir,
#     display=True
# )

**`DUSP1 Replica N 3hr 100nM time-sweep Partial Replica - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_N_slide1_BFmean_061825',
    'DUSP1_N_slide2_BFmean_061825',
    'DUSP1_N_slide3_BFmean_061825',
    'DUSP1_N_slide4_BFmean_061825',
    'DUSP1_N_slide5_BFmean_061825',
    'DUSP1_N_slide6_BFmean_061825',
    'DUSP1_N_slide7_BFmean_061825',      
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 50
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# ─── 5) main display ──────────────────────────────────────────────────────────
# build a fresh manager so the display code can find every HDF5
dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
display_manager = DUSP1DisplayManager(
    dm_disp,
    cell_level_results=SSITcellresults,
    spots=filtered_spots,
    clusters=filtered_clusters,
    cellprops=filtered_cellprops,
    removed_spots=removed_spots
)
display_manager.main_display()
dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_N_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults
)

# Time sweep for 100nM Dex
print("\n>>> Time sweep for 100nM Dex")
plotter.plot_time_sweep(
    dex_conc=100,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica N 3hr 100nM time-sweep Partial Replica - Full Dataset`**

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 1) Select & load the “N_Final2” analysis
# # ───────────────────────────────────────────────────────────────────────────────
# am = DUSP1AnalysisManager(location=loc,
#                           log_location=log_location,
#                           mac=mac)
# am.select_analysis('DUSP1_N_Final2')

# abs_threshold = 6
# mg_threshold  = 3

# spots_df    = am.select_datasets("spotresults",     dtype="dataframe")
# clusters_df = am.select_datasets("clusterresults",  dtype="dataframe")
# props_df    = am.select_datasets("cell_properties", dtype="dataframe")
# am.close()

# # ───────────────────────────────────────────────────────────────────────────────
# # 2) SNRAnalysis → DUSP1Measurement
# # ───────────────────────────────────────────────────────────────────────────────
# snr = SNRAnalysis(spots_df, props_df, clusters_df,
#                   abs_threshold=abs_threshold,
#                   mg_threshold=mg_threshold)
# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

# dusp = DUSP1Measurement(merged_spots_df,
#                         merged_clusters_df,
#                         merged_cellprops_df)
# cell_level_results = dusp.measure(abs_threshold=abs_threshold,
#                                   mg_threshold=mg_threshold)

# # ───────────────────────────────────────────────────────────────────────────────
# # 3) Prefix unique IDs (prefix=50 for “N”)
# # ───────────────────────────────────────────────────────────────────────────────
# max_id     = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))
# rep_prefix = 50
# prefix     = rep_prefix * (10 ** num_digits)

# for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
#     df['unique_cell_id'] += prefix

# max_spot_id  = merged_spots_df['unique_spot_id'].max()
# spot_prefix  = rep_prefix ** len(str(max_spot_id))
# merged_spots_df   ['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # ───────────────────────────────────────────────────────────────────────────────
# # 4) Apply filtering → get final tables + SSIT results
# # ───────────────────────────────────────────────────────────────────────────────
# filterer = DUSP1_filtering(method=method,
#                            abs_threshold=abs_threshold,
#                            mg_threshold=mg_threshold)
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
#     filterer.apply_all(spots=merged_spots_df,
#                        clusters=merged_clusters_df,
#                        cellprops=merged_cellprops_df)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5) Display image‐based overlays
# # ───────────────────────────────────────────────────────────────────────────────
# dm_disp = DUSP1AnalysisManager(location=loc,
#                                log_location=log_location,
#                                mac=mac)
# dm_disp.select_analysis('DUSP1_N_Final2')
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# # ───────────────────────────────────────────────────────────────────────────────
# # 6) Save only the final, post‐filter CSVs
# # ───────────────────────────────────────────────────────────────────────────────
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_N'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # ───────────────────────────────────────────────────────────────────────────────
# # 7) Post‐processing plot: single‐conc (100 nM) time sweep
# # ───────────────────────────────────────────────────────────────────────────────
# plotter = PostProcessingPlotter(
#     clusters_df=filtered_clusters,
#     cellprops_df=filtered_cellprops,
#     ssit_df=SSITcellresults
# )

# test_conc = 100
# print(f"\n>>> Time sweep @ {test_conc} nM Dex")
# plotter.plot_time_sweep(
#     dex_conc=test_conc,
#     save_dir=output_dir,
#     display=True
# )

**`DUSP1 Replica J 3hr time-concentration sweep R1 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_J_slide1_BFmean_061825',
    'DUSP1_J_slide2_BFmean_061825',
    'DUSP1_J_slide3_BFmean_061825',
    'DUSP1_J_slide4_BFmean_061825',
    'DUSP1_J_slide5_BFmean_061825',
    'DUSP1_J_slide6_BFmean_061825',
    'DUSP1_J_slide7_BFmean_061825',
    'DUSP1_J_slide8_BFmean_061825',
    'DUSP1_J_slide9_BFmean_061825',
    'DUSP1_J_slide10_BFmean_061825',
    'DUSP1_J_slide11_BFmean_061825',
    'DUSP1_J_slide12_BFmean_061825',
    'DUSP1_J_slide13_BFmean_061825',
    'DUSP1_J_slide14_BFmean_061825',
    'DUSP1_J_slide15_BFmean_061825',
    'DUSP1_J_slide16_BFmean_061825',
    'DUSP1_J_slide17_BFmean_061825',
    'DUSP1_J_slide18_BFmean_061825',
    'DUSP1_J_slide19_BFmean_061825',        
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 60
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# ─── 5) main display ──────────────────────────────────────────────────────────
# build a fresh manager so the display code can find every HDF5
dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
display_manager = DUSP1DisplayManager(
    dm_disp,
    cell_level_results=SSITcellresults,
    spots=filtered_spots,
    clusters=filtered_clusters,
    cellprops=filtered_cellprops,
    removed_spots=removed_spots
)
display_manager.main_display()
dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_J_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults
)

# Time + concentration sweeps for [0.3, 1, 10] nM Dex
print("\n>>> Time+Conc sweeps for [0.3, 1, 10] nM Dex")
plotter.plot_time_conc_sweep(
    conc_list=[0.3, 1, 10],
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica J 3hr time-concentration sweep R1 - By Day`**

In [ ]:
# # ─── 1) pull in all four days at once ──────────────────────────────────────────
# analysis_names = [
#     'DUSP1_J_day1_mean_BFmean',
#     'DUSP1_J_day2_mean_BFmean',
#     'DUSP1_J_day3_mean_BFmean',
#     'DUSP1_J_day4_mean_BFmean',
# ]

# # build and use the combined manager
# dm = make_combined_manager(loc, log_location, mac, analysis_names)

# spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
# clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
# props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
# dm.close()    # free all HDF5 handles

# print(f"Loaded and concatenated {len(analysis_names)} days:")
# print(f"  spots:   {spots_df.shape}")
# print(f"  clusters:{clusters_df.shape}")
# print(f"  props:   {props_df.shape}")

# # ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
# abs_threshold = 6
# mg_threshold  = 3

# snr = SNRAnalysis(spots_df, props_df, clusters_df,
#                   abs_threshold=abs_threshold,
#                   mg_threshold=mg_threshold)
# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

# dusp = DUSP1Measurement(merged_spots_df,
#                         merged_clusters_df,
#                         merged_cellprops_df)
# cell_level_results = dusp.measure(abs_threshold=abs_threshold,
#                                   mg_threshold=mg_threshold)

# # ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
# max_id     = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))
# rep_prefix = 60
# prefix     = rep_prefix * (10 ** num_digits)

# for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
#     df['unique_cell_id'] += prefix

# # spots
# max_spot_id  = merged_spots_df['unique_spot_id'].max()
# spot_prefix  = rep_prefix ** len(str(max_spot_id))
# merged_spots_df   ['unique_spot_id'] += spot_prefix

# # clusters
# max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # ─── 4) filtering ─────────────────────────────────────────────────────────────
# filterer = DUSP1_filtering(method=method,
#                            abs_threshold=abs_threshold,
#                            mg_threshold=mg_threshold)

# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
#     filterer.apply_all(spots=merged_spots_df,
#                        clusters=merged_clusters_df,
#                        cellprops=merged_cellprops_df)

# # ─── 5) main display ──────────────────────────────────────────────────────────
# # build a fresh manager so the display code can find every HDF5
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# # ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_J'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
# plotter = PostProcessingPlotter(
#     clusters_df=filtered_clusters,
#     cellprops_df=filtered_cellprops,
#     ssit_df=SSITcellresults
# )

# # Time + concentration sweeps for [0.3, 1, 10] nM Dex
# print("\n>>> Time+Conc sweeps for [0.3, 1, 10] nM Dex")
# plotter.plot_time_conc_sweep(
#     conc_list=[0.3, 1, 10],
#     save_dir=output_dir,
#     display=True
# )

**`DUSP1 Replica K 3hr time-concentration sweep R2 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_K_slide1_BFmean_061825',
    'DUSP1_K_slide2_BFmean_061825',
    'DUSP1_K_slide3_BFmean_061825',
    'DUSP1_K_slide4_BFmean_061825',
    'DUSP1_K_slide5_BFmean_061825',
    'DUSP1_K_slide6_BFmean_061825',
    'DUSP1_K_slide7_BFmean_061825',
    'DUSP1_K_slide8_BFmean_061825',
    'DUSP1_K_slide9_BFmean_061825',
    'DUSP1_K_slide10_BFmean_061825',
    'DUSP1_K_slide11_BFmean_061825',
    'DUSP1_K_slide12_BFmean_061825',
    'DUSP1_K_slide13_BFmean_061825',
    'DUSP1_K_slide14_BFmean_061825',
    'DUSP1_K_slide15_BFmean_061825',
    'DUSP1_K_slide16_BFmean_061825',
    'DUSP1_K_slide17_BFmean_061825',
    'DUSP1_K_slide18_BFmean_061825',
    'DUSP1_K_slide19_BFmean_061825',        
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 70
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) Filtering ───────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)
filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# ─── 5) Main display ───────────────────────────────────────────────────────────
dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
display_manager = DUSP1DisplayManager(
    dm_disp,
    cell_level_results=SSITcellresults,
    spots=filtered_spots,
    clusters=filtered_clusters,
    cellprops=filtered_cellprops,
    removed_spots=removed_spots
)
display_manager.main_display()
dm_disp.close()

# ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_K_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) Post‐processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults
)

print("\n>>> Time+Conc sweeps for [0.3, 1, 10] nM Dex")
plotter.plot_time_conc_sweep(
    conc_list=[0.3, 1, 10],
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica K 3hr time-concentration sweep R2 - By Day`**

In [ ]:
# # ─── 0) define your four‐day analysis names for replicate K ────────────────────
# analysis_names = [
#     'DUSP1_K_day1_mean_BFmean',
#     'DUSP1_K_day2_mean_BFmean',
#     'DUSP1_K_day3_mean_BFmean',
#     'DUSP1_K_day4_mean_BFmean',
# ]

# # ─── 1) Load & concat all days in one go ────────────────────────────────────────
# dm = make_combined_manager(loc, log_location, mac, analysis_names)
# spots_df    = dm.select_datasets("spotresults",     dtype="dataframe")
# clusters_df = dm.select_datasets("clusterresults",  dtype="dataframe")
# props_df    = dm.select_datasets("cell_properties", dtype="dataframe")
# dm.close()

# print(f"Loaded {len(analysis_names)} days for K: spots {spots_df.shape}, "
#       f"clusters {clusters_df.shape}, props {props_df.shape}")

# # ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
# abs_threshold = 6
# mg_threshold  = 3

# snr = SNRAnalysis(spots_df, props_df, clusters_df,
#                   abs_threshold=abs_threshold,
#                   mg_threshold=mg_threshold)
# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

# dusp = DUSP1Measurement(merged_spots_df,
#                         merged_clusters_df,
#                         merged_cellprops_df)
# cell_level_results = dusp.measure(abs_threshold=abs_threshold,
#                                   mg_threshold=mg_threshold)

# # ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
# max_id     = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))
# rep_prefix = 70
# prefix     = rep_prefix * (10 ** num_digits)

# for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
#     df['unique_cell_id'] += prefix

# max_spot_id  = merged_spots_df['unique_spot_id'].max()
# spot_prefix  = rep_prefix ** len(str(max_spot_id))
# merged_spots_df   ['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # ─── 4) Filtering ───────────────────────────────────────────────────────────────
# filterer = DUSP1_filtering(method=method,
#                            abs_threshold=abs_threshold,
#                            mg_threshold=mg_threshold)
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
#     filterer.apply_all(spots=merged_spots_df,
#                        clusters=merged_clusters_df,
#                        cellprops=merged_cellprops_df)

# # ─── 5) Main display ───────────────────────────────────────────────────────────
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# # ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_K'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # ─── 7) Post‐processing plots with PostProcessingPlotter ──────────────────────
# plotter = PostProcessingPlotter(
#     clusters_df=filtered_clusters,
#     cellprops_df=filtered_cellprops,
#     ssit_df=SSITcellresults
# )

# print("\n>>> Time+Conc sweeps for [0.3, 1, 10] nM Dex")
# plotter.plot_time_conc_sweep(
#     conc_list=[0.3, 1, 10],
#     save_dir=output_dir,
#     display=True
# )

**`DUSP1 Replica L 3hr 100nM time-concentration sweep R2 - By Slide`**

In [ ]:
# # ─── 1) pull in all slides  ──────────────────────────────────────────
# analysis_names = [
#     'DUSP1_L_slide1_BFmean_061825',
#     'DUSP1_L_slide2_BFmean_061825',
#     'DUSP1_L_slide3_BFmean_061825',
#     'DUSP1_L_slide4_BFmean_061825',
#     'DUSP1_L_slide5_BFmean_061825',
#     'DUSP1_L_slide6_BFmean_061825',
#     'DUSP1_L_slide7_BFmean_061825',
#     'DUSP1_L_slide8_BFmean_061825',
#     'DUSP1_L_slide9_BFmean_061825',
#     'DUSP1_L_slide10_BFmean_061825',
#     'DUSP1_L_slide11_BFmean_061825',
#     'DUSP1_L_slide12_BFmean_061825',
#     'DUSP1_L_slide13_BFmean_061825',
#     'DUSP1_L_slide14_BFmean_061825',
#     'DUSP1_L_slide15_BFmean_061825',
#     'DUSP1_L_slide16_BFmean_061825',
#     'DUSP1_L_slide17_BFmean_061825',
#     'DUSP1_L_slide18_BFmean_061825',       
# ]

# # build and use the combined manager
# dm = make_combined_manager(loc, log_location, mac, analysis_names)

# spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
# clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
# props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
# dm.close()    # free all HDF5 handles

# print(f"Loaded and concatenated {len(analysis_names)} days:")
# print(f"  spots:   {spots_df.shape}")
# print(f"  clusters:{clusters_df.shape}")
# print(f"  props:   {props_df.shape}")

# # ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
# abs_threshold = 6
# mg_threshold  = 3

# snr = SNRAnalysis(spots_df, props_df, clusters_df,
#                   abs_threshold=abs_threshold,
#                   mg_threshold=mg_threshold)
# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

# dusp = DUSP1Measurement(merged_spots_df,
#                         merged_clusters_df,
#                         merged_cellprops_df)
# cell_level_results = dusp.measure(abs_threshold=abs_threshold,
#                                   mg_threshold=mg_threshold)

# # ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
# max_id     = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))
# rep_prefix = 80
# prefix     = rep_prefix * (10 ** num_digits)

# for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
#     df['unique_cell_id'] += prefix

# max_spot_id  = merged_spots_df['unique_spot_id'].max()
# spot_prefix  = rep_prefix ** len(str(max_spot_id))
# merged_spots_df   ['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # ─── 4) Filtering ───────────────────────────────────────────────────────────────
# filterer = DUSP1_filtering(method=method,
#                            abs_threshold=abs_threshold,
#                            mg_threshold=mg_threshold)
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
#     filterer.apply_all(spots=merged_spots_df,
#                        clusters=merged_clusters_df,
#                        cellprops=merged_cellprops_df)

# # ─── 5) Main display ───────────────────────────────────────────────────────────
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# # ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_L'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # ─── 7) Post‐processing plots with PostProcessingPlotter ──────────────────────
# plotter = PostProcessingPlotter(
#     clusters_df=filtered_clusters,
#     cellprops_df=filtered_cellprops,
#     ssit_df=SSITcellresults
# )

# print("\n>>> Time+Conc sweeps for [0.3, 1, 10] nM Dex")
# plotter.plot_time_conc_sweep(
#     conc_list=[0.3, 1, 10],
#     save_dir=output_dir,
#     display=True
# )

**`DUSP1 Replica L 3hr 100nM time-concentration sweep R2 - By Group`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_L_day1_90th_BF90th',
    'DUSP1_L_day2_mean_BFmean',
    'DUSP1_L_day3_25th_BF25th',
    'DUSP1_L_day4_25th_BF25th',       
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 80
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) Filtering ───────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)
filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# ─── 5) Main display ───────────────────────────────────────────────────────────
dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
display_manager = DUSP1DisplayManager(
    dm_disp,
    cell_level_results=SSITcellresults,
    spots=filtered_spots,
    clusters=filtered_clusters,
    cellprops=filtered_cellprops,
    removed_spots=removed_spots
)
display_manager.main_display()
dm_disp.close()

# ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_L_BFmix'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) Post‐processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults
)

print("\n>>> Time+Conc sweeps for [0.3, 1, 10] nM Dex")
plotter.plot_time_conc_sweep(
    conc_list=[0.3, 1, 10],
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica L 3hr 100nM time-concentration sweep R2 - Full Dataset`**

In [ ]:
# # 1. Create an instance of the DUSP1AnalysisManager class.
# am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
# am.select_analysis('DUSP1_L_Final_061625')

# # Define thresholds
# abs_threshold = 6
# mg_threshold = 3

# # Load the datasets
# spots_df = am.select_datasets("spotresults", dtype="dataframe")
# clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
# props_df = am.select_datasets("cell_properties", dtype="dataframe")

# print(f"Data loaded and moving to SNRAnalysis...")
# # 2. Create an instance of the SNRAnalysis class.
# snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# print(f"SNR analysis complete, data merged and moving to DUSP1Measurement...")
# # 3. Create an instance of the DUSP1Measurement class.
# dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# # Process the data with a chosen threshold method
# cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# # Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# # Get number of digits in the max unique_cell_id
# max_id = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))

# # Calculate multiplier to add a '80' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
# rep_prefix = 80
# prefix = rep_prefix * (10 ** num_digits) 

# # Apply prefix to all related DataFrames
# merged_spots_df['unique_cell_id'] += prefix
# merged_clusters_df['unique_cell_id'] += prefix
# merged_cellprops_df['unique_cell_id'] += prefix
# cell_level_results['unique_cell_id'] += prefix

# # Repeat for unique_spot_id and unique_cluster_id
# max_spot_id = merged_spots_df['unique_spot_id'].max()
# spot_prefix = rep_prefix ** len(str(max_spot_id))
# merged_spots_df['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # Save the intermediate results
# rep_string = 'DUSP1_L'
# # intermediate_dir = save_dir
# # os.makedirs(intermediate_dir, exist_ok=True)
# # cell_level_results.to_csv(os.path.join(intermediate_dir, f"{rep_string}_cellresults_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_spots_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_Spots_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_clusters_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_Clusters_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_cellprops_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_CellProps_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # print(f"Intermediate results saved, moving to filtering...")


# # Initialize filtering object
# filterer = DUSP1_filtering(method=method, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# # Apply filtering and measurement
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = filterer.apply_all(
#     spots=merged_spots_df,
#     clusters=merged_clusters_df,
#     cellprops=merged_cellprops_df
# )

# print(f"Filtering complete, saving results...")
# # Save all results to CSV
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# SSITcellresults.to_csv(os.path.join(output_dir, f"{rep_string}_SSITcellresults_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_spots.to_csv(os.path.join(output_dir, f"{rep_string}_FinalSpots_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{rep_string}_FinalClusters_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{rep_string}_FinalCellProps_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)

# print(f"Results saved, moving to display...")
# # 4. Create an instance of the DUSP1DisplayManager class.
# display_manager = DUSP1DisplayManager(am, 
#                                       cell_level_results=SSITcellresults,
#                                       spots=filtered_spots,
#                                       clusters=filtered_clusters,
#                                       cellprops=filtered_cellprops,
#                                       removed_spots=removed_spots)
# # Run the main display function.
# display_manager.main_display()

# # 5. Create an instance of the PostProcessingDisplay class.
# post_processing_display = PostProcessingDisplay(spots_df=filtered_spots,
#                                                 clusters_df=filtered_clusters,
#                                                 cellprops_df=filtered_cellprops,
#                                                 ssit_cellresults_df= SSITcellresults)
# # Run the post-processing display function.
# post_processing_display.display_overview_plots(mode='time',
#                                                total= False,
#                                                display=True,
#                                                save_dir=output_dir)

**`DUSP1 Replica L 3hr time-concentration sweep R3 - By Day`**

In [ ]:
# # 0) Four‐day analysis names for DUSP1_L
# analysis_names_L = [
#     'DUSP1_L_day1_mean_BFmean',
#     'DUSP1_L_day2_mean_BFmean',
#     'DUSP1_L_day3_mean_BFmean',
#     'DUSP1_L_day4_mean_BFmean',
# ]

# # 1) Load & concatenate all four days
# dm_L = make_combined_manager(loc, log_location, mac, analysis_names_L)
# spots_df_L    = dm_L.select_datasets("spotresults",     dtype="dataframe")
# clusters_df_L = dm_L.select_datasets("clusterresults",  dtype="dataframe")
# props_df_L    = dm_L.select_datasets("cell_properties", dtype="dataframe")
# dm_L.close()

# print(f"Loaded {len(analysis_names_L)} days for L: "
#       f"spots {spots_df_L.shape}, clusters {clusters_df_L.shape}, props {props_df_L.shape}")

# # 2) SNRAnalysis → DUSP1Measurement
# abs_threshold = 6
# mg_threshold  = 3

# snr_L = SNRAnalysis(spots_df_L, props_df_L, clusters_df_L,
#                     abs_threshold=abs_threshold,
#                     mg_threshold=mg_threshold)
# merged_spots_L, merged_clusters_L, merged_cellprops_L = snr_L.get_results()

# dusp_L = DUSP1Measurement(merged_spots_L, merged_clusters_L, merged_cellprops_L)
# cell_level_results_L = dusp_L.measure(abs_threshold=abs_threshold,
#                                       mg_threshold=mg_threshold)

# # 3) Prefix unique IDs (prefix=80 for L replicate)
# max_id_L   = merged_cellprops_L['unique_cell_id'].max()
# digits_L   = len(str(max_id_L))
# rep_prefix = 80
# prefix_L   = rep_prefix * (10 ** digits_L)

# for df in (merged_spots_L, merged_clusters_L, merged_cellprops_L, cell_level_results_L):
#     df['unique_cell_id'] += prefix_L

# # also bump spot & cluster IDs
# max_spot_id_L    = merged_spots_L['unique_spot_id'].max()
# spot_prefix_L    = rep_prefix ** len(str(max_spot_id_L))
# merged_spots_L   ['unique_spot_id'] += spot_prefix_L

# max_cluster_id_L = merged_clusters_L['unique_cluster_id'].max()
# cluster_prefix_L = rep_prefix ** len(str(max_cluster_id_L))
# merged_clusters_L['unique_cluster_id'] += cluster_prefix_L

# # 4) Filtering → final tables + SSIT results
# filterer_L = DUSP1_filtering(method=method,
#                              abs_threshold=abs_threshold,
#                              mg_threshold=mg_threshold)
# filtered_spots_L, filtered_clusters_L, filtered_cellprops_L, SSIT_L, removed_spots_L = \
#     filterer_L.apply_all(
#         spots=merged_spots_L,
#         clusters=merged_clusters_L,
#         cellprops=merged_cellprops_L
#     )

# # 5) Image‐based overlay display
# dm_L_disp = make_combined_manager(loc, log_location, mac, analysis_names_L)
# display_mgr_L = DUSP1DisplayManager(
#     dm_L_disp,
#     cell_level_results=SSIT_L,
#     spots=filtered_spots_L,
#     clusters=filtered_clusters_L,
#     cellprops=filtered_cellprops_L,
#     removed_spots=removed_spots_L
# )
# display_mgr_L.main_display()
# dm_L_disp.close()

# # 6) Save only the final, post‐filter CSVs
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_L'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSIT_L           .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots_L .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters_L.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops_L.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # 7) Post‐processing plots for “L” replicate
# plotter_L = PostProcessingPlotter(
#     clusters_df=filtered_clusters_L,
#     cellprops_df=filtered_cellprops_L,
#     ssit_df=SSIT_L
# )

# # a) Time + concentration sweeps at [0.3, 1, 10] nM
# print("\n>>> [L] Time+Conc sweeps for [0.3, 1, 10] nM Dex")
# plotter_L.plot_time_conc_sweep(
#     conc_list=[0.3, 1, 10],
#     save_dir=output_dir,
#     display=True
# )

**`DUSP1 Replica G 75min Concentration sweep R1 - By Slide`**

In [ ]:
# # ─── 1) pull in all slides  ──────────────────────────────────────────
# analysis_names = [
#     'DUSP1_G_slide1_BF90th',
#     'DUSP1_G_slide2_BF90th',
#     'DUSP1_G_slide3_BF90th',
#     'DUSP1_G_slide4_BF90th',
#     'DUSP1_G_slide5_BF90th',
#     'DUSP1_G_slide6_BF90th',
#     'DUSP1_G_slide7_BF90th',
#     'DUSP1_G_slide8_BF90th',
#     'DUSP1_G_slide9_BF90th',      
# ]

# # build and use the combined manager
# dm = make_combined_manager(loc, log_location, mac, analysis_names)

# spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
# clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
# props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
# dm.close()    # free all HDF5 handles

# print(f"Loaded and concatenated {len(analysis_names)} days:")
# print(f"  spots:   {spots_df.shape}")
# print(f"  clusters:{clusters_df.shape}")
# print(f"  props:   {props_df.shape}")

# # ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
# abs_threshold = 6
# mg_threshold  = 3

# snr = SNRAnalysis(spots_df, props_df, clusters_df,
#                   abs_threshold=abs_threshold,
#                   mg_threshold=mg_threshold)
# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

# dusp = DUSP1Measurement(merged_spots_df,
#                         merged_clusters_df,
#                         merged_cellprops_df)
# cell_level_results = dusp.measure(abs_threshold=abs_threshold,
#                                   mg_threshold=mg_threshold)

# # ─── 3) Prefix unique IDs (prefix=90 for G replicate) ─────────────────────────
# max_id     = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))
# rep_prefix = 90
# prefix     = rep_prefix * (10 ** num_digits)

# for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
#     df['unique_cell_id'] += prefix

# max_spot_id  = merged_spots_df['unique_spot_id'].max()
# spot_prefix  = rep_prefix ** len(str(max_spot_id))
# merged_spots_df   ['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # ─── 4) Filtering ───────────────────────────────────────────────────────────────
# filterer = DUSP1_filtering(method=method,
#                            abs_threshold=abs_threshold,
#                            mg_threshold=mg_threshold)
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
#     filterer.apply_all(spots=merged_spots_df,
#                        clusters=merged_clusters_df,
#                        cellprops=merged_cellprops_df)

# # ─── 5) Main display ───────────────────────────────────────────────────────────
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# # ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_G'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # ─── 7) Post‐processing plots with PostProcessingPlotter ──────────────────────
# plotter = PostProcessingPlotter(
#     clusters_df=filtered_clusters,
#     cellprops_df=filtered_cellprops,
#     ssit_df=SSITcellresults
# )

# times = sorted(SSITcellresults['time'].unique())
# nonzero = [t for t in times if t != 0]
# if len(nonzero) != 1:
#     raise ValueError(f"Expected exactly one non-zero time for G sweep, found {nonzero}")
# timepoint = nonzero[0]

# print(f"\n>>> Concentration sweep at t={timepoint} min for G replicate")
# plotter.plot_conc_sweep(
#     timepoint=timepoint,
#     save_dir=output_dir,
#     display=True
# )

**`DUSP1 Replica G 75min Concentration sweep R1 - Full Dataset`**

In [ ]:
# # 1. Create an instance of the DUSP1AnalysisManager class.
# am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
# am.select_analysis('DUSP1_G_BFmax_BFmax_061825')

# # Define thresholds
# abs_threshold = 6
# mg_threshold = 3

# # Load the datasets
# spots_df = am.select_datasets("spotresults", dtype="dataframe")
# clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
# props_df = am.select_datasets("cell_properties", dtype="dataframe")

# print(f"Data loaded and moving to SNRAnalysis...")
# # 2. Create an instance of the SNRAnalysis class.
# snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# print(f"SNR analysis complete, data merged and moving to DUSP1Measurement...")
# # 3. Create an instance of the DUSP1Measurement class.
# dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# # Process the data with a chosen threshold method
# cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# # Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# # Get number of digits in the max unique_cell_id
# max_id = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))

# # Calculate multiplier to add a '80' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
# rep_prefix = 90
# prefix = rep_prefix * (10 ** num_digits) 

# # Apply prefix to all related DataFrames
# merged_spots_df['unique_cell_id'] += prefix
# merged_clusters_df['unique_cell_id'] += prefix
# merged_cellprops_df['unique_cell_id'] += prefix
# cell_level_results['unique_cell_id'] += prefix

# # Repeat for unique_spot_id and unique_cluster_id
# max_spot_id = merged_spots_df['unique_spot_id'].max()
# spot_prefix = rep_prefix ** len(str(max_spot_id))
# merged_spots_df['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # Save the intermediate results
# rep_string = 'DUSP1_G_BFmax'
# # intermediate_dir = save_dir
# # os.makedirs(intermediate_dir, exist_ok=True)
# # cell_level_results.to_csv(os.path.join(intermediate_dir, f"{rep_string}_cellresults_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_spots_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_Spots_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_clusters_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_Clusters_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_cellprops_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_CellProps_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # print(f"Intermediate results saved, moving to filtering...")

# # Initialize filtering object
# filterer = DUSP1_filtering(method=method, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# # Apply filtering and measurement
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = filterer.apply_all(
#     spots=merged_spots_df,
#     clusters=merged_clusters_df,
#     cellprops=merged_cellprops_df
# )

# print(f"Filtering complete, saving results...")
# # Save all results to CSV
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# SSITcellresults.to_csv(os.path.join(output_dir, f"{rep_string}_SSITcellresults_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_spots.to_csv(os.path.join(output_dir, f"{rep_string}_FinalSpots_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{rep_string}_FinalClusters_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{rep_string}_FinalCellProps_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)

# print(f"Results saved, moving to display...")
# # 4. Create an instance of the DUSP1DisplayManager class.
# display_manager = DUSP1DisplayManager(am, 
#                                       cell_level_results=SSITcellresults,
#                                       spots=filtered_spots,
#                                       clusters=filtered_clusters,
#                                       cellprops=filtered_cellprops,
#                                       removed_spots=removed_spots)
# # Run the main display function.
# display_manager.main_display()

# # ─── 7) Post‐processing plots with PostProcessingPlotter ──────────────────────
# plotter = PostProcessingPlotter(
#     clusters_df=filtered_clusters,
#     cellprops_df=filtered_cellprops,
#     ssit_df=SSITcellresults
# )

# times = sorted(SSITcellresults['time'].unique())
# nonzero = [t for t in times if t != 0]
# if len(nonzero) != 1:
#     raise ValueError(f"Expected exactly one non-zero time for G sweep, found {nonzero}")
# timepoint = nonzero[0]

# print(f"\n>>> Concentration sweep at t={timepoint} min for G replicate")
# plotter.plot_conc_sweep(
#     timepoint=timepoint,
#     save_dir=output_dir,
#     display=True
# )

**`DUSP1 Replica G 75min concentration sweep R1 - By Day`**

In [ ]:
# 0) Define the three‐day analysis names for DUSP1_G
analysis_names_G = [
    'DUSP1_G_day1_BF75th',
    'DUSP1_G_day2_BF75th',
    'DUSP1_G_day3_75th',
]

# 1) Load & concatenate all three days in one pass
dm_G = make_combined_manager(loc, log_location, mac, analysis_names_G)
spots_df_G    = dm_G.select_datasets("spotresults",     dtype="dataframe")
clusters_df_G = dm_G.select_datasets("clusterresults",  dtype="dataframe")
props_df_G    = dm_G.select_datasets("cell_properties", dtype="dataframe")
dm_G.close()

print(f"Loaded {len(analysis_names_G)} days for G: "
      f"spots {spots_df_G.shape}, clusters {clusters_df_G.shape}, props {props_df_G.shape}")

# 2) SNRAnalysis → DUSP1Measurement
abs_threshold = 6
mg_threshold  = 3

snr_G = SNRAnalysis(spots_df_G, props_df_G, clusters_df_G,
                    abs_threshold=abs_threshold,
                    mg_threshold=mg_threshold)
merged_spots_G, merged_clusters_G, merged_cellprops_G = snr_G.get_results()

dusp_G = DUSP1Measurement(merged_spots_G, merged_clusters_G, merged_cellprops_G)
cell_level_results_G = dusp_G.measure(abs_threshold=abs_threshold,
                                      mg_threshold=mg_threshold)

# 3) Prefix unique IDs (prefix=90 for G replicate)
max_id_G   = merged_cellprops_G['unique_cell_id'].max()
digits_G   = len(str(max_id_G))
rep_prefix = 90
prefix_G   = rep_prefix * (10 ** digits_G)

for df in (merged_spots_G, merged_clusters_G, merged_cellprops_G, cell_level_results_G):
    df['unique_cell_id'] += prefix_G

# also prefix spot and cluster IDs
max_spot_id_G    = merged_spots_G['unique_spot_id'].max()
spot_prefix_G    = rep_prefix ** len(str(max_spot_id_G))
merged_spots_G   ['unique_spot_id'] += spot_prefix_G

max_cluster_id_G = merged_clusters_G['unique_cluster_id'].max()
cluster_prefix_G = rep_prefix ** len(str(max_cluster_id_G))
merged_clusters_G['unique_cluster_id'] += cluster_prefix_G

# 4) Filtering
filterer_G = DUSP1_filtering(method=method,
                             abs_threshold=abs_threshold,
                             mg_threshold=mg_threshold)
filtered_spots_G, filtered_clusters_G, filtered_cellprops_G, SSIT_G, removed_spots_G = \
    filterer_G.apply_all(
        spots=merged_spots_G,
        clusters=merged_clusters_G,
        cellprops=merged_cellprops_G
    )

# 5) Image overlay display
dm_disp_G = make_combined_manager(loc, log_location, mac, analysis_names_G)
display_manager_G = DUSP1DisplayManager(
    dm_disp_G,
    cell_level_results=SSIT_G,
    spots=filtered_spots_G,
    clusters=filtered_clusters_G,
    cellprops=filtered_cellprops_G,
    removed_spots=removed_spots_G
)
display_manager_G.main_display()
dm_disp_G.close()

# 6) Save only the final, post‐filter CSVs
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_G_BF75th'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSIT_G           .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots_G .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters_G.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops_G.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# 7) Post‐processing concentration sweep plot
#    auto-detect the single nonzero timepoint for this experiment
plotter_G = PostProcessingPlotter(
    clusters_df=filtered_clusters_G,
    cellprops_df=filtered_cellprops_G,
    ssit_df=SSIT_G
)
times = sorted(SSIT_G['time'].unique())
nonzero = [t for t in times if t != 0]
if len(nonzero) != 1:
    raise ValueError(f"Expected exactly one non-zero time for G sweep, found {nonzero}")
timepoint_G = nonzero[0]

print(f"\n>>> Concentration sweep at t={timepoint_G} min for G replicate")
plotter_G.plot_conc_sweep(
    timepoint=timepoint_G,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica H 75min concentration sweep R2 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_H_slide1_BFmean_061825',
    'DUSP1_H_slide2_BFmean_061825',
    'DUSP1_H_slide3_BFmean_061825',
    'DUSP1_H_slide4_BFmean_061825',
    'DUSP1_H_slide5_BFmean_061825',
    'DUSP1_H_slide6_BFmean_061825',
    'DUSP1_H_slide7_BFmean_061825',
    'DUSP1_H_slide8_BFmean_061825',    
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 110
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) Filtering ───────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)
filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# ─── 5) Main display ───────────────────────────────────────────────────────────
dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
display_manager = DUSP1DisplayManager(
    dm_disp,
    cell_level_results=SSITcellresults,
    spots=filtered_spots,
    clusters=filtered_clusters,
    cellprops=filtered_cellprops,
    removed_spots=removed_spots
)
display_manager.main_display()
dm_disp.close()

# ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_H_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) Post‐processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults
)

times = sorted(SSITcellresults['time'].unique())
nonzero = [t for t in times if t != 0]
if len(nonzero) != 1:
    raise ValueError(f"Expected exactly one non-zero time for H sweep, found {nonzero}")
timepoint = nonzero[0]

print(f"\n>>> Concentration sweep at t={timepoint} min for H replicate")
plotter.plot_conc_sweep(
    timepoint=timepoint,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica H 75min concentration sweep R2 - Full Dataset`**

In [ ]:
# # 1. Create an instance of the DUSP1AnalysisManager class.
# am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
# am.select_analysis('DUSP1_H_Final')

# # Define thresholds
# abs_threshold = 6
# mg_threshold = 3

# # Load the datasets
# spots_df = am.select_datasets("spotresults", dtype="dataframe")
# clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
# props_df = am.select_datasets("cell_properties", dtype="dataframe")

# print(f"Data loaded and moving to SNRAnalysis...")
# # 2. Create an instance of the SNRAnalysis class.
# snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# print(f"SNR analysis complete, data merged and moving to DUSP1Measurement...")
# # 3. Create an instance of the DUSP1Measurement class.
# dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# # Process the data with a chosen threshold method
# cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# # Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# # Get number of digits in the max unique_cell_id
# max_id = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))

# # Calculate multiplier to add a '80' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
# rep_prefix = 110
# prefix = rep_prefix * (10 ** num_digits) 

# # Apply prefix to all related DataFrames
# merged_spots_df['unique_cell_id'] += prefix
# merged_clusters_df['unique_cell_id'] += prefix
# merged_cellprops_df['unique_cell_id'] += prefix
# cell_level_results['unique_cell_id'] += prefix

# # Repeat for unique_spot_id and unique_cluster_id
# max_spot_id = merged_spots_df['unique_spot_id'].max()
# spot_prefix = rep_prefix ** len(str(max_spot_id))
# merged_spots_df['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # Save the intermediate results
# rep_string = 'DUSP1_H'
# # intermediate_dir = save_dir
# # os.makedirs(intermediate_dir, exist_ok=True)
# # cell_level_results.to_csv(os.path.join(intermediate_dir, f"{rep_string}_cellresults_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_spots_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_Spots_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_clusters_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_Clusters_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_cellprops_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_CellProps_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # print(f"Intermediate results saved, moving to filtering...")

# # Initialize filtering object
# filterer = DUSP1_filtering(method=method, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# # Apply filtering and measurement
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = filterer.apply_all(
#     spots=merged_spots_df,
#     clusters=merged_clusters_df,
#     cellprops=merged_cellprops_df
# )

# print(f"Filtering complete, saving results...")
# # Save all results to CSV
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# SSITcellresults.to_csv(os.path.join(output_dir, f"{rep_string}_SSITcellresults_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_spots.to_csv(os.path.join(output_dir, f"{rep_string}_FinalSpots_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{rep_string}_FinalClusters_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{rep_string}_FinalCellProps_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)

# print(f"Results saved, moving to display...")
# # 4. Create an instance of the DUSP1DisplayManager class.
# display_manager = DUSP1DisplayManager(am, 
#                                       cell_level_results=SSITcellresults,
#                                       spots=filtered_spots,
#                                       clusters=filtered_clusters,
#                                       cellprops=filtered_cellprops,
#                                       removed_spots=removed_spots)
# # Run the main display function.
# display_manager.main_display()

# # 5. Create an instance of the PostProcessingDisplay class.
# post_processing_display = PostProcessingDisplay(spots_df=filtered_spots,
#                                                 clusters_df=filtered_clusters,
#                                                 cellprops_df=filtered_cellprops,
#                                                 ssit_cellresults_df= SSITcellresults)
# # Run the post-processing display function.
# post_processing_display.display_overview_plots(mode='conc',
#                                                total= False,
#                                                display=True,
#                                                save_dir=output_dir)

**`DUSP1 Replica H 75min concentration sweep R2 - By Day`**

In [ ]:
# # ─── 0) Define the two‐day analysis names for DUSP1_H ───────────────────────────
# analysis_names_H = [
#     'DUSP1_H_day1_mean_BFmean',
#     'DUSP1_H_day2_mean_BFmean',
# ]

# # ─── 1) Load & concatenate both days ──────────────────────────────────────────
# dm_H = make_combined_manager(loc, log_location, mac, analysis_names_H)
# spots_df_H    = dm_H.select_datasets("spotresults",     dtype="dataframe")
# clusters_df_H = dm_H.select_datasets("clusterresults",  dtype="dataframe")
# props_df_H    = dm_H.select_datasets("cell_properties", dtype="dataframe")
# dm_H.close()

# print(f"Loaded {len(analysis_names_H)} days for H: "
#       f"spots {spots_df_H.shape}, clusters {clusters_df_H.shape}, props {props_df_H.shape}")

# # ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
# abs_threshold = 6
# mg_threshold  = 3

# snr_H = SNRAnalysis(spots_df_H, props_df_H, clusters_df_H,
#                     abs_threshold=abs_threshold,
#                     mg_threshold=mg_threshold)
# merged_spots_H, merged_clusters_H, merged_cellprops_H = snr_H.get_results()

# dusp_H = DUSP1Measurement(merged_spots_H, merged_clusters_H, merged_cellprops_H)
# cell_level_results_H = dusp_H.measure(abs_threshold=abs_threshold,
#                                       mg_threshold=mg_threshold)

# # ─── 3) Prefix unique IDs (prefix=110 for H replicate) ─────────────────────────
# max_id_H   = merged_cellprops_H['unique_cell_id'].max()
# digits_H   = len(str(max_id_H))
# rep_prefix = 110
# prefix_H   = rep_prefix * (10 ** digits_H)

# for df in (merged_spots_H, merged_clusters_H, merged_cellprops_H, cell_level_results_H):
#     df['unique_cell_id'] += prefix_H

# max_spot_id_H    = merged_spots_H['unique_spot_id'].max()
# spot_prefix_H    = rep_prefix ** len(str(max_spot_id_H))
# merged_spots_H   ['unique_spot_id'] += spot_prefix_H

# max_cluster_id_H = merged_clusters_H['unique_cluster_id'].max()
# cluster_prefix_H = rep_prefix ** len(str(max_cluster_id_H))
# merged_clusters_H['unique_cluster_id'] += cluster_prefix_H

# # ─── 4) Filtering ───────────────────────────────────────────────────────────────
# filterer_H = DUSP1_filtering(method=method,
#                              abs_threshold=abs_threshold,
#                              mg_threshold=mg_threshold)
# filtered_spots_H, filtered_clusters_H, filtered_cellprops_H, SSIT_H, removed_spots_H = \
#     filterer_H.apply_all(
#         spots=merged_spots_H,
#         clusters=merged_clusters_H,
#         cellprops=merged_cellprops_H
#     )

# # ─── 5) Image overlay display ─────────────────────────────────────────────────
# dm_disp_H = make_combined_manager(loc, log_location, mac, analysis_names_H)
# display_manager_H = DUSP1DisplayManager(
#     dm_disp_H,
#     cell_level_results=SSIT_H,
#     spots=filtered_spots_H,
#     clusters=filtered_clusters_H,
#     cellprops=filtered_cellprops_H,
#     removed_spots=removed_spots_H
# )
# display_manager_H.main_display()
# dm_disp_H.close()

# # ─── 6) Save only the final, post‐filter CSVs ───────────────────────────────────
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_H'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSIT_H           .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots_H .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters_H.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops_H.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # ─── 7) Post‐processing concentration sweep ────────────────────────────────────
# plotter_H = PostProcessingPlotter(
#     clusters_df=filtered_clusters_H,
#     cellprops_df=filtered_cellprops_H,
#     ssit_df=SSIT_H
# )

# # auto‐detect the single non‐zero timepoint
# times_H = sorted(SSIT_H['time'].unique())
# nonzero_H = [t for t in times_H if t != 0]
# if len(nonzero_H) != 1:
#     raise ValueError(f"Expected one non-zero timepoint for H sweep, got {nonzero_H}")
# timepoint_H = nonzero_H[0]

# print(f"\n>>> Concentration sweep at t={timepoint_H} min for H replicate")
# plotter_H.plot_conc_sweep(
#     timepoint=timepoint_H,
#     save_dir=output_dir,
#     display=True
# )

**`DUSP1 Replica I 75min concentration sweep R3 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_I_slide1_BFmean_061825',
    'DUSP1_I_slide2_BFmean_061825',
    'DUSP1_I_slide3_BFmean_061825',
    'DUSP1_I_slide4_BFmean_061825',
    'DUSP1_I_slide5_BFmean_061825',
    'DUSP1_I_slide6_BFmean_061825',
    'DUSP1_I_slide7_BFmean_061825',
    'DUSP1_I_slide8_BFmean_061825',
    'DUSP1_I_slide9_BFmean_061825',      
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 120
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) Filtering ───────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)
filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# ─── 5) Main display ───────────────────────────────────────────────────────────
dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
display_manager = DUSP1DisplayManager(
    dm_disp,
    cell_level_results=SSITcellresults,
    spots=filtered_spots,
    clusters=filtered_clusters,
    cellprops=filtered_cellprops,
    removed_spots=removed_spots
)
display_manager.main_display()
dm_disp.close()

# ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_I_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) Post‐processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults
)

times = sorted(SSITcellresults['time'].unique())
nonzero = [t for t in times if t != 0]
if len(nonzero) != 1:
    raise ValueError(f"Expected exactly one non-zero time for I sweep, found {nonzero}")
timepoint = nonzero[0]

print(f"\n>>> Concentration sweep at t={timepoint} min for I replicate")
plotter.plot_conc_sweep(
    timepoint=timepoint,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica I 75min concentration sweep R3 - Full Dataset`**

In [ ]:
# # 1. Create an instance of the DUSP1AnalysisManager class.
# am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
# am.select_analysis('DUSP1_I_Final')

# # Define thresholds
# abs_threshold = 6
# mg_threshold = 3

# # Load the datasets
# spots_df = am.select_datasets("spotresults", dtype="dataframe")
# clusters_df = am.select_datasets("clusterresults", dtype="dataframe")
# props_df = am.select_datasets("cell_properties", dtype="dataframe")

# print(f"Data loaded and moving to SNRAnalysis...")
# # 2. Create an instance of the SNRAnalysis class.
# snr_df = SNRAnalysis(spots_df, props_df, clusters_df, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# merged_spots_df, merged_clusters_df, merged_cellprops_df = snr_df.get_results()

# print(f"SNR analysis complete, data merged and moving to DUSP1Measurement...")
# # 3. Create an instance of the DUSP1Measurement class.
# dusp = DUSP1Measurement(merged_spots_df, merged_clusters_df, merged_cellprops_df)

# # Process the data with a chosen threshold method
# cell_level_results = dusp.measure(abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# # Add replica level unique IDs for 'unique_cell_id', 'unique_spot_id', and 'unique_cluster_id'
# # Get number of digits in the max unique_cell_id
# max_id = merged_cellprops_df['unique_cell_id'].max()
# num_digits = len(str(max_id))

# # Calculate multiplier to add a '80' followed by the right number of zeroes - prefix is specific for each experiment (e.g., repD:1, repE:2, etc.)
# rep_prefix = 120
# prefix = rep_prefix * (10 ** num_digits) 

# # Apply prefix to all related DataFrames
# merged_spots_df['unique_cell_id'] += prefix
# merged_clusters_df['unique_cell_id'] += prefix
# merged_cellprops_df['unique_cell_id'] += prefix
# cell_level_results['unique_cell_id'] += prefix

# # Repeat for unique_spot_id and unique_cluster_id
# max_spot_id = merged_spots_df['unique_spot_id'].max()
# spot_prefix = rep_prefix ** len(str(max_spot_id))
# merged_spots_df['unique_spot_id'] += spot_prefix

# max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
# cluster_prefix = rep_prefix ** len(str(max_cluster_id))
# merged_clusters_df['unique_cluster_id'] += cluster_prefix

# # Save the intermediate results
# rep_string = 'DUSP1_I'
# # intermediate_dir = save_dir
# # os.makedirs(intermediate_dir, exist_ok=True)
# # cell_level_results.to_csv(os.path.join(intermediate_dir, f"{rep_string}_cellresults_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_spots_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_Spots_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_clusters_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_Clusters_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # merged_cellprops_df.to_csv(os.path.join(intermediate_dir, f"{rep_string}_CellProps_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# # print(f"Intermediate results saved, moving to filtering...")

# # Initialize filtering object
# filterer = DUSP1_filtering(method=method, abs_threshold=abs_threshold, mg_threshold=mg_threshold)

# # Apply filtering and measurement
# filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = filterer.apply_all(
#     spots=merged_spots_df,
#     clusters=merged_clusters_df,
#     cellprops=merged_cellprops_df
# )

# print(f"Filtering complete, saving results...")
# # Save all results to CSV
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# SSITcellresults.to_csv(os.path.join(output_dir, f"{rep_string}_SSITcellresults_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_spots.to_csv(os.path.join(output_dir, f"{rep_string}_FinalSpots_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_clusters.to_csv(os.path.join(output_dir, f"{rep_string}_FinalClusters_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)
# filtered_cellprops.to_csv(os.path.join(output_dir, f"{rep_string}_FinalCellProps_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}.csv"), index=False)

# print(f"Results saved, moving to display...")
# # 4. Create an instance of the DUSP1DisplayManager class.
# display_manager = DUSP1DisplayManager(am, 
#                                       cell_level_results=SSITcellresults,
#                                       spots=filtered_spots,
#                                       clusters=filtered_clusters,
#                                       cellprops=filtered_cellprops,
#                                       removed_spots=removed_spots)
# # Run the main display function.
# display_manager.main_display()

# # 5. Create an instance of the PostProcessingDisplay class.
# post_processing_display = PostProcessingDisplay(spots_df=filtered_spots,
#                                                 clusters_df=filtered_clusters,
#                                                 cellprops_df=filtered_cellprops,
#                                                 ssit_cellresults_df= SSITcellresults)
# # Run the post-processing display function.
# post_processing_display.display_overview_plots(mode='conc',
#                                                total= False,
#                                                display=True,
#                                                save_dir=output_dir)

**`DUSP1 Replica I 75min concentration sweep R3 - By Day`**

In [ ]:
# # 0) Define the three-day analysis names for DUSP1_I
# analysis_names_I = [
#     'DUSP1_I_day1_mean_BFmean',
#     'DUSP1_I_day2_mean_BFmean',
#     'DUSP1_I_day3_mean_BFmean',
# ]

# # 1) Load & concatenate all three days in one pass
# dm_I = make_combined_manager(loc, log_location, mac, analysis_names_I)
# spots_df_I    = dm_I.select_datasets("spotresults",     dtype="dataframe")
# clusters_df_I = dm_I.select_datasets("clusterresults",  dtype="dataframe")
# props_df_I    = dm_I.select_datasets("cell_properties", dtype="dataframe")
# dm_I.close()

# print(f"Loaded {len(analysis_names_I)} days for I: "
#       f"spots {spots_df_I.shape}, clusters {clusters_df_I.shape}, props {props_df_I.shape}")

# # 2) SNRAnalysis → DUSP1Measurement
# abs_threshold = 6
# mg_threshold  = 3

# snr_I = SNRAnalysis(spots_df_I, props_df_I, clusters_df_I,
#                     abs_threshold=abs_threshold,
#                     mg_threshold=mg_threshold)
# merged_spots_I, merged_clusters_I, merged_cellprops_I = snr_I.get_results()

# dusp_I = DUSP1Measurement(merged_spots_I, merged_clusters_I, merged_cellprops_I)
# cell_level_results_I = dusp_I.measure(abs_threshold=abs_threshold,
#                                       mg_threshold=mg_threshold)

# # 3) Prefix unique IDs (prefix=120 for I replicate)
# max_id_I   = merged_cellprops_I['unique_cell_id'].max()
# digits_I   = len(str(max_id_I))
# rep_prefix = 120
# prefix_I   = rep_prefix * (10 ** digits_I)

# for df in (merged_spots_I, merged_clusters_I, merged_cellprops_I, cell_level_results_I):
#     df['unique_cell_id'] += prefix_I

# max_spot_id_I    = merged_spots_I['unique_spot_id'].max()
# spot_prefix_I    = rep_prefix ** len(str(max_spot_id_I))
# merged_spots_I   ['unique_spot_id'] += spot_prefix_I

# max_cluster_id_I = merged_clusters_I['unique_cluster_id'].max()
# cluster_prefix_I = rep_prefix ** len(str(max_cluster_id_I))
# merged_clusters_I['unique_cluster_id'] += cluster_prefix_I

# # 4) Filtering → final tables + SSIT results
# filterer_I = DUSP1_filtering(method=method,
#                              abs_threshold=abs_threshold,
#                              mg_threshold=mg_threshold)
# filtered_spots_I, filtered_clusters_I, filtered_cellprops_I, SSIT_I, removed_spots_I = \
#     filterer_I.apply_all(
#         spots=merged_spots_I,
#         clusters=merged_clusters_I,
#         cellprops=merged_cellprops_I
#     )

# # 5) Image overlay display
# dm_disp_I = make_combined_manager(loc, log_location, mac, analysis_names_I)
# display_manager_I = DUSP1DisplayManager(
#     dm_disp_I,
#     cell_level_results=SSIT_I,
#     spots=filtered_spots_I,
#     clusters=filtered_clusters_I,
#     cellprops=filtered_cellprops_I,
#     removed_spots=removed_spots_I
# )
# display_manager_I.main_display()
# dm_disp_I.close()

# # 6) Save only the final, post-filter CSVs
# output_dir = save_dir
# os.makedirs(output_dir, exist_ok=True)
# rep_string = 'DUSP1_I'
# base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

# SSIT_I            .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
# filtered_spots_I  .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
# filtered_clusters_I.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
# filtered_cellprops_I.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

# print("Saved final results to:", output_dir)

# # 7) Post-processing concentration sweep plot
# plotter_I = PostProcessingPlotter(
#     clusters_df=filtered_clusters_I,
#     cellprops_df=filtered_cellprops_I,
#     ssit_df=SSIT_I
# )

# # auto-detect the single non-zero timepoint
# times_I = sorted(SSIT_I['time'].unique())
# nonzero_I = [t for t in times_I if t != 0]
# if len(nonzero_I) != 1:
#     raise ValueError(f"Expected one non-zero timepoint for I sweep, got {nonzero_I}")
# timepoint_I = nonzero_I[0]

# print(f"\n>>> Concentration sweep at t={timepoint_I} min for I replicate")
# plotter_I.plot_conc_sweep(
#     timepoint=timepoint_I,
#     save_dir=output_dir,
#     display=True
# )